<a href="https://colab.research.google.com/github/sajjad73hajizade/shirazU-learn/blob/main/Calculate_Real_Temp_landsat7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rasterio

In [3]:
import rasterio
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et
from tifffile import imsave

In [ ]:
dataset_main = rasterio.open('tehran.tif')
band3_red = dataset_main.read(3)
band4_nir = dataset_main.read(4)
band6_temp = dataset_main.read(6)

Red = band3_red
NearIR = band4_nir
Surface_Temp = band6_temp

In [5]:
ML = 0
AL = 0
tr = et.parse('LE07_L1TP_160041_20221215_20221216_02_RT_MTL.xml')
root = tr.getroot()
for Landsat_info in root.iter('RADIANCE_MULT_BAND_6_VCID_1'):
    ML = float(Landsat_info.text)

for Landsat_info in root.iter('RADIANCE_ADD_BAND_6_VCID_1'):
    AL = float(Landsat_info.text)

for Landsat_info in root.iter('K1_CONSTANT_BAND_6_VCID_1'):
    K1 = float(Landsat_info.text)

for Landsat_info in root.iter('K2_CONSTANT_BAND_6_VCID_1'):
    K2 = float(Landsat_info.text)

In [ ]:
TOA = Surface_Temp * ML + AL
BT = (K2 / (np.log(K1 / TOA) + 1)) -273.15
BT = np.nan_to_num(BT)

In [ ]:
#NDVI = (nir – red) / (nir + red)
NDVI = (NearIR - Red) / (NearIR + Red)
NDVI = np.nan_to_num(NDVI)

numrows = len(NDVI)
numcols = len(NDVI[0])

In [8]:
i = 0
j = 0

while i < numrows:
    while j < numcols:
        ndvi = NDVI[i][j]
        if ndvi > 0.2 and ndvi < 0.5:
            emisi = (0.004 * ((ndvi - 0.2)/(0.5 - 0.2))) + 0.9
            BT[i][j] = BT[i][j] / emisi
        elif ndvi > 0.5:
            emisi = 0.99
            BT[i][j] = BT[i][j] / emisi
        elif ndvi < 0.2:
            emisi = 0.97
            BT[i][j] = BT[i][j] / emisi
        j += 1
    i += 1



In [9]:
print(BT)

[[  0.           0.           0.         ... -17.30502641 -17.88045516
  -17.88045516]
 [  0.           0.           0.         ... -17.13197614 -17.70165061
  -17.70165061]
 [  0.           0.           0.         ... -16.84856488 -15.72421649
  -15.72421649]
 ...
 [  0.           0.36285885   0.36285885 ...  -9.00498041 -14.33905383
  -14.33905383]
 [  0.           0.36285885   0.36285885 ...  -9.00498041 -14.33905383
  -14.33905383]
 [  0.           0.59775403   0.59775403 ... -14.61432867 -17.70165061
  -17.70165061]]
